In [ ]:
#r "nuget:CardanoSharp.Wallet,0.2.2"
#r "nuget:Flurl"
#r "nuget:Flurl.Http"

Installed Packages CardanoSharp.Wallet, 0.2.2 Flurl, 3.0.2 Flurl.Http, 3.2.0

# Wallet Basic Overview

In this section, we're going to review how to:
 - Generate a New Mnemonic
 - Restore a Mnemonic
 - Get the Root Key for the Mnemonic
 - Derive Private and Public Keys
 - Generate an Address


## Generate a New Mnemonic 

Lets start simple. Here were just going to generate a new mnemonic.

In [ ]:
using CardanoSharp.Wallet;
using CardanoSharp.Wallet.Enums;
using CardanoSharp.Wallet.Models.Keys;
using CardanoSharp.Wallet.Models;
using CardanoSharp.Wallet.Models.Transactions;
using CardanoSharp.Wallet.Models.Derivations;
using CardanoSharp.Wallet.Extensions;
using CardanoSharp.Wallet.Extensions.Models;
using CardanoSharp.Wallet.Extensions.Models.Transactions;

IKeyService keyService = new KeyService();
Mnemonic mnemonic = keyService.Generate(24, WordLists.English);

// Console.WriteLine($"Words: {mnemonic.Words}");
// Console.WriteLine($"Entropy: {mnemonic.Entropy.ToStringHex()}");

MasterNodeDerivation masterNode = mnemonic.GetMasterNode();
IAccountNodeDerivation accountNode = masterNode
    .Derive(PurposeType.Shelley)
    .Derive(CoinType.Ada)
    .Derive(0);
accountNode.SetPublicKey();

//Deriving from Public Key
var paymentPub = accountNode.PublicKey
    .Derive(RoleType.ExternalChain)
    .Derive(0);
var stakePub = accountNode.PublicKey
    .Derive(RoleType.Staking)
    .Derive(0);


//Deriving from Private Key
var paymentNode = accountNode.PrivateKey
    .Derive(RoleType.ExternalChain)
    .Derive(0);
paymentNode.SetPublicKey();
var stakeNode = accountNode.PrivateKey
    .Derive(RoleType.Staking)
    .Derive(0);
stakeNode.SetPublicKey();

Transaction tx = new Transaction();
tx.TransactionBody = new TransactionBody() {
    TransactionInputs = new List<TransactionInput>(),
    TransactionOutputs = new List<TransactionOutput>()
};
tx.TransactionWitnessSet = new TransactionWitnessSet() {
    VKeyWitnesses = new List<VKeyWitness>()
};
tx.AuxiliaryData = new AuxiliaryData() {
    Metadata = new Dictionary<int, object>()
    {
        { 1234, new { name = "simple message" } }
    } 
};
var cborSignedTx = tx.GetCBOR();
Console.WriteLine(cborSignedTx);
var serializedSignedTx = tx.Serialize();
Console.WriteLine(serializedSignedTx.ToStringHex());



    // PrivateKey rootKey = mnemonic.GetRootKey();
    // IAddressService addressService = new AddressService();

    // // This path will give us our Stake Key on index 0
    // var stakePath = $"m/1852'/1815'/0'/2/0";
    // // The stakePrv is another Tuple with the Private Key and Chain Code
    // var stakePrv = rootKey.Derive(stakePath);
    // // Get the Public Key from the Stake Private Key
    // var stakePub = stakePrv.GetPublicKey(false); 
    
    // for(var i = 0; i < 20; i++){
    //     // This path will give us our Payment Key on index 0
    //     string paymentPath = $"m/1852'/1815'/0'/0/{i}";
    //     // The paymentPrv is another Tuple with the Private Key and Chain Code
    //     PrivateKey paymentPrv = rootKey.Derive(paymentPath); //skey
    //     // Get the Public Key from the Payment Private Key
    //     PublicKey paymentPub = paymentPrv.GetPublicKey(false); //vkey
    
    //     var baseAddr = addressService.GetAddress(
    //         paymentPub, 
    //         stakePub, 
    //         NetworkType.Testnet, 
    //         AddressType.Base);
    //     Console.WriteLine($"Delegation Address: {baseAddr}");
    // }



[{2: 0, 7: h'8DC8A798A1DA0E2A6DF17E66B10A49B5047133DD4DAAE2686EF1F73369D3FA16'}, {}, [{1234: {"name": "simple message"}}, []]]
System.Byte[]


## Restoring a Mnemonic

Now that we have our mnemonic, lets pretend we are loading it into a new wallet.

Here we will take our words and "Restore" them. This will produce the same object in the end.

In [ ]:
Mnemonic restoredMnemonic = keyService.Restore(mnemonic.Words, WordLists.English);

Console.WriteLine($"Words are the same: {mnemonic.Words == restoredMnemonic.Words}");
Console.WriteLine($"Entropy are the same: {mnemonic.Entropy.ToStringHex() == restoredMnemonic.Entropy.ToStringHex()}");

## Get the Root Key for the Mnemonic

Generating the Root Key is the first step into our Hierarchical Deterministic(HD) Wallet. This key will let us derive any key we want and we will be able to do it over and over again... Deterministic. 

As users of wallets, we are used to seeing the famous Mnemonic Words, but we also see above that there is something called Entropy. Entropy is the bytes version of your words, and this is the piece that we use to generate our Root Key. So just note that what is really going on here is Words -> Entropy -> Root Key.

In [ ]:
using CardanoSharp.Wallet.Extensions.Models;
using CardanoSharp.Wallet.Encoding;

PrivateKey rootKey = mnemonic.GetRootKey();
string rootEncoded = Bech32.Encode(rootKey.Key, "root_xsk");
Console.WriteLine($"Bech32 Encoded Root Private Key: {rootEncoded}");

A few points about this root key before we move on to deriving various Private and Public Keys.
 -  This is the root's private key, but the private key is more specifically an extended private key. This is defined in BIP 32.
    -  Learn more about BIP 32: [Click Here](https://en.bitcoin.it/wiki/BIP_0032)
 -  When encoding, we are following the CIP 005 prefix rules. This is why we put `root_xsk`. 
    -  Learn more about CIP 0005: [Click Here](https://github.com/cardano-foundation/CIPs/blob/master/CIP-0005/CIP-0005.md)


## Derive Private and Public Keys

Now that we can Create/Retore Mnemonics and get a Root Key, it's time to start deriving some Private and Public keys. This where we dive into the Hierarchical piece of the HD wallet. 

In [ ]:
IAddressService addressService = new AddressService();

// This path will give us our Stake Key on index 0
var stakePath = $"m/1852'/1815'/0'/2/0";
// The stakePrv is another Tuple with the Private Key and Chain Code
var stakePrv = rootKey.Derive(stakePath);
// Get the Public Key from the Stake Private Key
var stakePub = stakePrv.GetPublicKey(false); 

for(var i = 0; i < 20; i++){
    // This path will give us our Payment Key on index 0
    string paymentPath = $"m/1852'/1815'/0'/0/{i}";
    // The paymentPrv is another Tuple with the Private Key and Chain Code
    PrivateKey paymentPrv = rootKey.Derive(paymentPath); //skey
    // Get the Public Key from the Payment Private Key
    PublicKey paymentPub = paymentPrv.GetPublicKey(false); //vkey

    var baseAddr = addressService.GetAddress(
        paymentPub, 
        stakePub, 
        NetworkType.Testnet, 
        AddressType.Base);
    Console.WriteLine($"Delegation Address: {baseAddr}");
}
